In [1]:
import networkx as nx
import jarvis
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -U plotly
!pip install pygraphviz --install-option="--include-path=/usr/local/include/graphviz/" --install-option="--library-path=/usr/local/lib/graphviz"

Requirement already up-to-date: plotly in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages
Requirement already up-to-date: six in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (from plotly)
Requirement already up-to-date: decorator>=4.0.6 in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (from plotly)
Requirement already up-to-date: requests in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (from plotly)
Requirement already up-to-date: pytz in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (from plotly)
Requirement already up-to-date: nbformat>=4.2 in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (from plotly)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (from requests->plotly)
Requirement already up-to-date: certifi>=2017.4.17 in /Users/jegonzal/anaconda3/envs/jarvis/lib/python3.6/site-packages (

In [3]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [5]:
from crawl import tr_crawl
tr_crawler = jarvis.Action(tr_crawl)
training_tweets = jarvis.Artifact('training_tweets.csv', tr_crawler)

from clean import clean
tr_cleaner = jarvis.Action(clean, [training_tweets])
clean_training_tweets = jarvis.Artifact('clean_training_tweets.pkl', tr_cleaner)

from train_model import train
trainer = jarvis.Action(train, [clean_training_tweets])
intermediary = jarvis.Artifact('intermediary.pkl', trainer)

from crawl import te_crawl
te_crawler = jarvis.Action(te_crawl)
testing_tweets = jarvis.Artifact('testing_tweets.csv', te_crawler)

te_cleaner = jarvis.Action(clean, [testing_tweets])
clean_testing_tweets = jarvis.Artifact('clean_testing_tweets.pkl', te_cleaner)

from test_model import test
tester = jarvis.Action(test, [intermediary, clean_testing_tweets])
model_accuracy = jarvis.Artifact('model_accuracy.txt', tester)



In [6]:
model_accuracy.pull()

In [7]:
import plotly.offline as py
import plotly.graph_objs as go

def plot_graph(g):
    # Graph layout could be tweaked a little: 
    # https://networkx.github.io/documentation/stable/reference/drawing.html#module-networkx.drawing.nx_pydot
    pos = nx.nx_agraph.graphviz_layout(g, prog="dot")
    
    node_trace = go.Scatter(
        x=[pos[v][0] for v in g.nodes()],
        y=[pos[v][1] for v in g.nodes()],
        text=[g.node[v]['text'].replace("\n", "<br>") for v in g.nodes()],
        mode='markers+text',
        textposition="right",
        hoverinfo='text',
        marker=dict(
            size=10,
            color=[
                g.node[v]['shape'].replace("box", "red").replace("ellipse", "blue") for v in g.nodes()  
            ],
            symbol=[
                g.node[v]['shape'].replace("box", "square").replace("ellipse", "circle") for v in g.nodes()
        ]))
    
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in g.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]
    
    fig = go.Figure(data=[node_trace, edge_trace],
             layout=go.Layout(
                title='Plot1',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False,
                               range=[0, max([pos[k][0] for k in pos])*1.3]),
                yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False)))
    py.iplot(fig)


In [8]:

## Make Plot
g = model_accuracy.to_graph()
plot_graph(g)